In [ ]:
# SENTINEL2_BandArithmetic -----------------------------------------------------
# end to end implementation of the following indexes

# BSI: The Bare Soil Index
# FDI: The Floating Debris Index (see: Finding Plastic Patches in Coastal Waters using Optical Satellite Data, Lauren Biermann et al)
# NBR: The Normalized Burn Ratio
# NDVI: The Normalized Vegegation Index
# NDWI: The Normalized Difference Water Index
# TCI: True Color Image (this task directly from the tci image in newer sentinell2 payloads)

# February - May 2021, RTS
#-------------------------------------------------------------------------------
# Notes:
# Ensure the file sentinel2_helper.py is in the correct locatation (see below)
# Assumes you already have downloaded ++ a single sentinel2 product ++ with 'Sentinel2_getdata'
# and that this download resides in the 'sentinel' directory (see below)
# Assumes you are using the same geojson file here as in 'Sentinel2_getdata'

# Process:
# First band operation: run all cells from top to bottom
# Subsequent band operations: continue from 'operations for NDWI, NDVI, NBR, BSI, FDI, TCI'
# (you can omit the previous steps as the requisite variables persist in the jupyter notebook session)
# Last band operation: run the final cell to clean up intermediate files and 
# archive the sentinal2 download, if desired.
#-------------------------------------------------------------------------------

In [ ]:
import sys
#clear all variables (just in case)
sys.modules[__name__].__dict__.clear()
import os, sys, time, random, itertools, shutil

#mount the google drive
from google.colab import drive
drive.mount('/content/drive')

#maximize screen real estate
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#all the required installations here
!pip install rasterio --upgrade
!pip install rioxarray --upgrade
!pip install geopandas --upgrade
!pip install earthpy --upgrade

print('\n\nFINISHED installing rasterio rioxarray, geopandas, earthpy....')

In [3]:
# variables specific to your CoLab Jupyter notebook setup ----------------------

start = '/content/drive/MyDrive/your directory/'

# update your path to include loctions of helper files, specific to your setup
codepath = start + 'code/'
sys.path.append(codepath)

#set other directories (create them manually on your drive)
datapath = start + 'data/'
searchpath = datapath + 'sentinel/'
archivepath = datapath + 'archive/'

# place a valid geojson file in the .../data directory
map = 'bali_kuta_beaches.geojson'
geojsonpath = datapath

In [4]:
# all imports here
import rasterio
import rioxarray
from rasterio.mask import mask
from rasterio.plot import show
from skimage.io import imsave
import matplotlib.pyplot as plt
import numpy
import geopandas
import earthpy.plot as ep
import warnings
warnings.filterwarnings('ignore')

#  homebrew
from sentinel2_helper import *

In [ ]:
# <start here for the first pass or if you have remove itermediate files with the last module on this page>
# these next steps assume you have already downloaded zipped satellite image package and that is resides in your searchpath
unpack(searchpath)

In [ ]:
imageroot = get_imageroot(searchpath)
print('Path to band data: ', imageroot)

In [ ]:
# <continue here if you already processed an operation and have not yet archived the downloaded sentinel package with the last module on this page>
# prepare and perform operations for NDWI, NDVI, NBR, BSI, FDI, TCI

band_operation = 'TCI'
#-------------------------------------------------------------------------------
prefix = band_operation.lower()
img_type = prefix + '.tif'
img_ccr_type = prefix + '_4326.tif'

#-------------------------------------------------------------------------------
if (band_operation == 'NDVI'):
  band4=rasterio.open(imageroot + 'B04.jp2')      #red
  band8=rasterio.open(imageroot + 'B08.jp2')      #nir
  red = band4.read()
  nir = band8.read()

  refband = band4
  result_img = ndvi(red, nir)
  convert_jp2_tif(searchpath, img_type, refband, result_img)

#-------------------------------------------------------------------------------
elif (band_operation == 'NDWI'):
  band3=rasterio.open(imageroot + 'B03.jp2')      #green
  band8=rasterio.open(imageroot + 'B08.jp2')      #nir
  green = band3.read()
  nir = band8.read()

  refband = band3
  result_img  = ndwi(green, nir) 
  convert_jp2_tif(searchpath, img_type, refband, result_img)

#-------------------------------------------------------------------------------
elif (band_operation == 'NBR'):
  band12=rasterio.open(imageroot + 'B12.jp2' )    #SWIR 20m
  band8A=rasterio.open(imageroot + 'B8A.jp2')     #nir 20m
  swir = band12.read()
  nir = band8A.read()

  refband = band8A
  result_img  = nbr(nir, swir) 
  convert_jp2_tif(searchpath, img_type, refband, result_img)

#-------------------------------------------------------------------------------
elif (band_operation == 'BSI'):
  b11 = 'B11.jp2'                                 #swir 1610nm, 20m
  scale_factor = 2                                #upsample from 20m res to 10m

  resampled_b11 = resample(searchpath, imageroot, b11, scale_factor)

  band2=rasterio.open(imageroot + 'B02.jp2')      #blue, 10m
  band8=rasterio.open(imageroot + 'B08.jp2')      #nir, 10m
  band4=rasterio.open(imageroot + 'B04.jp2')      #red, 10m
  sw = rasterio.open(searchpath + resampled_b11)

  blue = band2.read()
  nir = band8.read()
  red = band4.read()
  swir = sw.read()
  
  refband = band2
  result_img  = bsi(red, blue, nir, swir) 
  convert_jp2_tif(searchpath, img_type, refband, result_img)

#-------------------------------------------------------------------------------
elif (band_operation == 'FDI'):
  b11 = 'B11.jp2'                                 #swir 1610nm, 20m
  b6 = 'B06.jp2'                                  #rededge2 740nm
  scale_factor = 2                                #upsample from 20m res to 10m

  resampled_b11 = resample(searchpath, imageroot, b11, scale_factor)
  resampled_b6 = resample(searchpath, imageroot, b6, scale_factor)
  band8 = rasterio.open(imageroot + 'B08.jp2')    #nir
  nir = band8.read()
  sw = rasterio.open(searchpath + resampled_b11)
  swir = sw.read()
  red = rasterio.open(searchpath + resampled_b6)
  rededge = red.read()

  lambda_nir = 832.8                              #nm
  lambda_red = 664.6                              #nm
  lambda_swir = 1613.7                            #nm

  refband = band8
  result_img = fdi(nir, rededge, swir, lambda_nir, lambda_red, lambda_swir)
  convert_jp2_tif(searchpath, img_type, refband, result_img)

#-------------------------------------------------------------------------------
elif (band_operation == 'TCI'):
  tci = get_tci(searchpath)
  convert_tci_tif(tci, searchpath, img_type)
#-------------------------------------------------------------------------------
else:
  print(' ... pick from one of the available operations: NDWI, NDVI, NBR, BSI, FDI, TCI')

print(band_operation + ' tasks have completed... continue to next cells')

In [ ]:
#convert coordinate system
target_epsg = "EPSG:4326"
convert_ccr(searchpath, img_type, target_epsg, img_ccr_type)
print('CRS conversion complete')

In [ ]:
#check the geojson file (extent of the area we are interested in)
gdf = geopandas.read_file(geojsonpath + map)
gdf.plot();

In [ ]:
#apply mask, rename image, display and save
current_4326_img = rasterio.open(searchpath + img_ccr_type) 
#apply the geojson file produced mask
masked, mask_transform = mask(dataset=current_4326_img, shapes=gdf.geometry, crop=True)
newname = create_name(prefix, imageroot, geojsonpath, map)

#display and save
if ('tci' in img_ccr_type):
  show(masked, transform=mask_transform);
  temp = rasterio.plot.reshape_as_image(masked)
  imsave(datapath + newname +'.jpeg', temp)

else:
  colormap =   'gray'                                                               #'gray'    #'RdYlGn'  #'viridis' ... others
  ax = ep.plot_bands(masked, cmap=colormap, cols=1, vmin=-1, vmax=1, cbar=False)    #cbar=False OR True
  ax.figure.savefig(datapath + newname + '.jpeg')
  ax.figure.show()

print(newname + ' saved to searchpath')

In [ ]:
#clean up
REUSE = True
ARCHIVE = True
archive(searchpath, archivepath, REUSE, ARCHIVE)